<a href="https://colab.research.google.com/github/satani99/practical_deep_learning_for_coders/blob/main/fast_ai_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [2]:
files = get_text_files(path, folders=['train', 'test', 'unsup'])

In [3]:
txt = files[0].open().read(); txt[:75]

'WTF!! Do any of his books/movies end in a happy ending?? The Notebook was g'

In [4]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#186) ['WTF','!','!','Do','any','of','his','books','/','movies','end','in','a','happy','ending','?','?','The','Notebook','was','good','...','but','sheesh',',','enough','with','the','depressing','endings'...]


In [5]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

In [6]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#196) ['xxbos','xxup','wtf','!','!','xxmaj','do','any','of','his','books','/','movies','end','in','a','happy','ending','?','?','xxmaj','the','xxmaj','notebook','was','good','…','but','sheesh',',','enough'...]


In [7]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [8]:
??replace_rep

In [9]:
coll_repr(tkn('&copy; Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

In [10]:
txts = L(o.open().read() for o in files[:2000])

In [11]:
def subword(sz):
  sp = SubwordTokenizer(vocab_sz=sz)
  sp.setup(txts)
  return ' '.join(first(sp([txt]))[:40])

In [12]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 


In [13]:
subword(1000)

'▁W T F ! ! ▁Do ▁any ▁of ▁his ▁book s / mo vi es ▁end ▁in ▁a ▁ha pp y ▁end ing ? ? ▁The ▁N o te bo ok ▁was ▁good ... b ut ▁she e sh ,'

In [14]:
subword(200)

'▁ W T F ! ! ▁ D o ▁ an y ▁of ▁ h i s ▁b o o k s / m o vi es ▁ en d ▁in ▁a ▁ ha p p y ▁ en d'

In [15]:
subword(10000)

"▁W T F !! ▁Do ▁any ▁of ▁his ▁books / movie s ▁end ▁in ▁a ▁happy ▁ending ?? ▁The ▁Not e book ▁was ▁good ... but ▁she es h , ▁enough ▁with ▁the ▁depressing ▁ending s ▁already . ▁I '"

In [16]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#196) ['xxbos','xxup','wtf','!','!','xxmaj','do','any','of','his','books','/','movies','end','in','a','happy','ending','?','?','xxmaj','the','xxmaj','notebook','was','good','…','but','sheesh',',','enough'...]


In [17]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#196) ['xxbos','xxup','wtf','!','!','xxmaj','do','any','of','his'...]

In [18]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

"(#2216) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

In [19]:
nums = num(toks)[:20]; nums

TensorText([   2,    7,    0,   62,   62,    8,   76,  154,   14,   36, 1616,
             134,  132,  118,   17,   13,  510,  469,   80,   80])

In [20]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxup xxunk ! ! xxmaj do any of his books / movies end in a happy ending ? ?'

In [21]:
nums200 = toks200.map(num)

In [22]:
dl = LMDataLoader(nums200)

In [23]:
x, y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [24]:
' ' .join(num.vocab[o] for o in x[0][:20])

'xxbos xxup xxunk ! ! xxmaj do any of his books / movies end in a happy ending ? ?'

In [25]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxup xxunk ! ! xxmaj do any of his books / movies end in a happy ending ? ? xxmaj'

In [26]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

In [27]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj the xxmaj two xxmaj thrones is the third and last installment in the xxmaj prince of xxmaj persia trilogy that started three years ago with the astonishing xxmaj sands of xxmaj time , a game that turned the popular 2d platform series into a complete and spectacular 3d adventure . xxmaj jumping , swinging , grasping , hanging off ledges , solving puzzles and graceful combat made up for one of the best games i ever played .","xxmaj the xxmaj two xxmaj thrones is the third and last installment in the xxmaj prince of xxmaj persia trilogy that started three years ago with the astonishing xxmaj sands of xxmaj time , a game that turned the popular 2d platform series into a complete and spectacular 3d adventure . xxmaj jumping , swinging , grasping , hanging off ledges , solving puzzles and graceful combat made up for one of the best games i ever played . xxmaj"
1,"xxmaj levine and xxmaj martin xxmaj starr ) . \n\n "" freaks and xxmaj geeks "" presents a realistic view of high school life , unlike the glamorized , soap opera high schools of shows like "" the xxup oc "" . xxmaj and this show has heart . xxmaj it has all of the emotions you 'd expect from a show about teenagers . xxmaj there is n't always a happy ending , just like in life . xxmaj","levine and xxmaj martin xxmaj starr ) . \n\n "" freaks and xxmaj geeks "" presents a realistic view of high school life , unlike the glamorized , soap opera high schools of shows like "" the xxup oc "" . xxmaj and this show has heart . xxmaj it has all of the emotions you 'd expect from a show about teenagers . xxmaj there is n't always a happy ending , just like in life . xxmaj but"


In [28]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()


In [30]:
learn.fit_one_cycle(1, 2e-2)

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time


KeyboardInterrupt: ignored

In [ ]:
learn.save('1epoch')

In [ ]:
learn.unfreeze()
leamrn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save_encoder('finetuned')

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)]
print("\n".join(preds))

In [32]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab), CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

In [33]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj some have praised xxunk xxmaj lost xxmaj xxunk as a xxmaj disney adventure for adults . i do n't think so -- at least not for thinking adults . \n\n xxmaj this script suggests a beginning as a live - action movie , that struck someone as the type of crap you can not sell to adults anymore . xxmaj the "" crack staff "" of many older adventure movies has been done well before , ( think xxmaj the xxmaj dirty xxmaj dozen ) but xxunk represents one of the worse films in that motif . xxmaj the characters are weak . xxmaj even the background that each member trots out seems stock and awkward at best . xxmaj an xxup md / xxmaj medicine xxmaj man , a tomboy mechanic whose father always wanted sons , if we have not at least seen these before ,",neg
2,"xxbos xxmaj warning : xxmaj does contain spoilers . \n\n xxmaj open xxmaj your xxmaj eyes \n\n xxmaj if you have not seen this film and plan on doing so , just stop reading here and take my word for it . xxmaj you have to see this film . i have seen it four times so far and i still have n't made up my mind as to what exactly happened in the film . xxmaj that is all i am going to say because if you have not seen this film , then stop reading right now . \n\n xxmaj if you are still reading then i am going to pose some questions to you and maybe if anyone has any answers you can email me and let me know what you think . \n\n i remember my xxmaj grade 11 xxmaj english teacher quite well . xxmaj",pos


In [34]:
nums_samp = toks200[:10].map(num)

In [35]:
nums_samp.map(len)

(#10) [196,489,260,124,276,464,131,310,1209,195]

In [37]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5,
                                metrics=accuracy).to_fp16()

In [38]:
learn.fit_one_cycle(1, 2e-2)

/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: ignored